# EEG Preprocessing

## Library Imports

In [36]:
# pip install pandas
# pip install numpy
# pip install mne
# pip install scikit-learn

In [37]:
pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/40/77/91f92b2fddbd14201bf36cd0c0e7279f1501a88e7a00ef11261c4b95bb7a/scikit_learn-1.4.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7a35d1cefa9a1bcfb/threadpoolctl-3.5.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 660.6 kB/s eta 0:00:16
   ---------------------------------------- 0.0/10.6 MB 487.6 kB/s eta 0:00:22
   ---------------------------------------- 0.1/10.6 MB 581.0 


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
import numpy as np
import mne
import datetime
import os

import matplotlib as mpl
mpl.use('QtAgg')
import matplotlib.pyplot as plt

## EEG Data Import

In [49]:
eeg_data_path = "Data/UnicornRecorder_20240508_155309.csv"
otree_data_path = "Data/all_apps_wide-2024-05-08.csv"
participant_id = "e7rtpxhj"

In [50]:
# Load only the first 8 columns from the CSV (the EEG channels)
raw_data = pd.read_csv(eeg_data_path, usecols=range(8))

In [51]:
# Specify our Channel Setup
channel_names = ['F3', 'Fz', 'F4', 'T3', 'C3', 'C4', 'T4', 'Pz']
channel_types = ['eeg'] * 8 

# Transpose to shape (n_channels, n_samples)
data = raw_data.values.T  

# Create an Info object, necessary for creating a Raw object
info = mne.create_info(ch_names=channel_names, sfreq=250, ch_types=channel_types)  # sfreq is the sampling frequency

# Create the RawArray object
raw = mne.io.RawArray(data, info)

# Set Montage
raw.set_montage('standard_1020')

Creating RawArray with float64 data, n_channels=8, n_times=650288
    Range : 0 ... 650287 =      0.000 ...  2601.148 secs
Ready.


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [52]:
raw.compute_psd().plot()

Effective window size : 8.192 (s)
Plotting power spectral density (dB=True).


C:\Users\fabiostano\AppData\Local\Temp\ipykernel_14492\4108630121.py:1: FutureWarning: The value of `amplitude='auto'` will be removed in MNE 1.8.0, and the new default will be `amplitude=False`.
  raw.compute_psd().plot()


<MNELineFigure size 1000x350 with 2 Axes>

### Timestamps

This is an example for just using the preference as event labels for each song start. Depending on the Research Question you might have to change this. For example: Create just 2 labels for <= 4 and > 4; OR use the arousal or pleasure indications as labels.

NOTE: We just have the starting time in seconds. This is very unprecise but should work if we leave out the first and last window of each song.

In [53]:
# Extract the filename without extension
filename = os.path.splitext(os.path.basename(eeg_data_path))[0]

# Extract the date and time from the filename
date_str = filename.split('_')[1]
time_str = filename.split('_')[2]

# Combine into a single datetime object
start_datetime = datetime.datetime.strptime(date_str + time_str, '%Y%m%d%H%M%S')

In [54]:
# Load the data
events_df = pd.read_csv(otree_data_path)

# Filter for the specific participant
participant_events = events_df[events_df['participant.code'] == participant_id]

# Columns for event timestamps and preference ratings
event_time_columns = [f"Music_Discovery.{i}.player.time_start" for i in range(1, 41)]
preference_columns = [f"Music_Discovery.{i}.player.preference" for i in range(1, 41)]

# Extract event timestamps and preferences
event_timestamps_unix = participant_events[event_time_columns].values[0]
preferences = participant_events[preference_columns].values[0]

# Convert Unix timestamps (in ms) to datetime objects
event_times = [datetime.datetime.fromtimestamp(ts / 1000.0) for ts in event_timestamps_unix]

# Calculate the time differences in seconds relative to EEG start time
event_onsets = [(et - start_datetime).total_seconds() for et in event_times]

# Create descriptions incorporating preference ratings
descriptions = [int(pref) for pref in preferences]

# Create annotations
durations = [0] * 40  # Assuming instant events
annotations = mne.Annotations(onset=event_onsets, duration=durations, description=descriptions)

# Add annotations to the raw object
raw.set_annotations(annotations)

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [55]:
raw.plot()

<MNEBrowseFigure size 1920x1129 with 4 Axes>

Channels marked as bad:
none


## EEG Preprocessing

- High-pass filtering at 1 Hz and low-pass filtering at 40 Hz.
- This can be played with. 
- I guess HP 0.1Hz - 1Hz; LP 30Hz - 100H could be feasible.

In [56]:
raw.filter(l_freq=0.5, h_freq=60.0)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 1651 samples (6.604 s)



Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.50 Hz
Lowpass,60.00 Hz


- Apply a notch filter at 50 Hz
- This could be improved by more sophisticated methods like ZapLine


In [57]:
raw.notch_filter(freqs=50.0)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)



Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.50 Hz
Lowpass,60.00 Hz


### ICA
This has to be done by visual inspection, could maybe be automated.

In [58]:
from mne.preprocessing import ICA

# pick some channels that clearly show blinks
artifact_picks = mne.pick_channels(ch_names=raw.info['ch_names'], include=['Fz', 'F3', 'F4'])
raw.plot(order=artifact_picks, n_channels=len(artifact_picks))

ica = mne.preprocessing.ICA(n_components=8, max_iter="auto", random_state=97)
ica.fit(raw)

explained_var_ratio = ica.get_explained_variance_ratio(raw)
for channel_type, ratio in explained_var_ratio.items():
  print(
      f"Fraction of {channel_type} variance explained by all components: " f"{ratio}"
  )

raw.load_data()
ica.plot_sources(raw)
ica.plot_components()

Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Fitting ICA took 2.3s.
Fraction of eeg variance explained by all components: 1.0
Creating RawArray with float64 data, n_channels=8, n_times=650288
    Range : 0 ... 650287 =      0.000 ...  2601.148 secs
Ready.
Channels marked as bad:
none


<MNEFigure size 975x496 with 8 Axes>

ICA001 sieht ein bisschen nach blinzeln aus, ist aber mMn nicht clean genug um excluded zu werden. Ich vermute wir würden auch andere Daten dadurch verlieren. Man könnte es probieren das würde dann so gehen:

In [48]:
# Set ICA Component To Exclude
#ica.exclude = [1]
#ica.apply(raw)

# Narrow Bandpass Filter
# raw = raw.filter(l_freq=0.1, h_freq=30, fir_design='firwin')

Applying ICA to Raw instance
    Transforming to ICA space (8 components)
    Zeroing out 1 ICA component
    Projecting back using 8 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 8251 samples (33.004 s)



## Epoching Data

Hier muss jetzt der Durchgängige Datenstream in Epochen geschnitten werden. Z.B.: ausgehend von jedem event, werden zuerst 3 sekunden ignoriert (bis das Lied wirkt, und um Synchronisationsprobleme zu vermeiden), und dann 10 x 2s windows geschnitten. Dann kann man alle windows die zu einer Klasse gehören (z.B. alle wo die bewertung über 4 war = Like-Epochen), in ein "Epochs" Objekt packen. 